In [1]:
pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-34_8cne4
  Running command git clone --filter=blob:none --quiet https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-34_8cne4
  Resolved https://github.com/ContinualAI/avalanche.git to commit d752103e838babda2e22e2771d5630561c9496c0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using

In [2]:
import avalanche
avalanche.__version__

'0.5.0a'

### **Data**

In [3]:
import librosa as lr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import IPython.display as ipd
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle as pk
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [4]:
dataset='/content/drive/MyDrive/dataset/augmented_dataset'
pd.DataFrame(os.listdir(dataset),columns=['Files'])

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/synthetic-speech-commands-dataset/augmented_dataset/augmented_dataset'

In [6]:
def count(path):
    size=[]
    for file in os.listdir(path):
        size.append(len(os.listdir(os.path.join(path,file))))
    return pd.DataFrame(size,columns=['Number Of Sample'],index=os.listdir(path))

In [ ]:
tr=count(dataset)
tr

In [ ]:
plt.figure(figsize=(10,10))
plt.pie(x='Number Of Sample',labels=os.listdir(dataset),autopct ='%1.1f%%',data=tr)
plt.title('Distribution Of Data In Train',fontsize=20)
plt.show()

In [ ]:
#bird Audio
ipd.Audio('/content/drive/MyDrive/dataset/augmented_dataset/augmented_dataset/bird/1019.wav')

In [ ]:
#yes Audio
ipd.Audio('/content/drive/MyDrive/dataset/augmented_dataset/augmented_dataset/yes/1014.wav')

In [ ]:
def load(path):
    data=[]
    label=[]
    sample=[]
    for file in os.listdir(path):
        path_=os.path.join(path,file)
        for fil in os.listdir(path_):
            data_contain,sample_rate=lr.load(os.path.join(path_,fil) ,sr=16000)
            data.append(data_contain)
            sample.append(sample_rate)
            label.append(file)
    return data,label,sample

In [ ]:
data,label,sample=load(dataset)
df=pd.DataFrame()
df['Label'],df['sample']=label,sample
df

### **Audio Feature Extraction**

In [ ]:
#waveform
def waveform(data,sr,label):
    plt.figure(figsize=(14, 5))
    lr.display.waveshow(data, sr=sr)
    plt.suptitle(label)
    plt.title('Waveform plot')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
#..........................................................

#MFCC features
def mfcc(data, sr):
    mfccs = lr.feature.mfcc(y=data, sr=sr)
    return np.mean(mfccs), mfccs
def mfcc_v(mfccs,label):
    plt.figure(figsize=(10, 4))
    lr.display.specshow(mfccs, x_axis='time')
    plt.colorbar()
    plt.title('MFCC')
    plt.suptitle(label)
#..........................................................

#Mel-spectrogram
def Mel(data, sr):
    mel_spec = lr.feature.melspectrogram(y=data, sr=sr)
    return np.mean(mel_spec), mel_spec
def mel_v(mel_spec,label,sr):
    # Convert to decibel scale
    mel_spec_db = lr.power_to_db(mel_spec, ref=np.max)
    # Visualize Mel-spectrogram
    plt.figure(figsize=(10, 4))
    lr.display.specshow(mel_spec_db, x_axis='time', y_axis='mel', sr=sr)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-spectrogram')
    plt.suptitle(label)
#..........................................................

#zero_crossing_rate
def zero_crossing(data,sr):
    # Compute zero-crossing rate
    zcr = lr.feature.zero_crossing_rate(data)
    # Print average zero-crossing rate
    avg_zcr = sum(zcr[0])/len(zcr[0])
    print("Average zero-crossing rate:", avg_zcr)
    return zcr

def zero_crossing_v(zcr,label,data,sr):
    time = lr.times_like(zcr)
    # Create waveform plot
    plt.figure(figsize=(14, 5))
    lr.display.waveshow(data, sr=sr, alpha=0.5)
    plt.plot(time, zcr[0], color='r')
    plt.title('Zero-crossing rate')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.suptitle(label)

In [ ]:
#waveform data[0]

waveform(data[0],sample[0],label[0])
plt.legend()

In [ ]:
#MFCC data[0]

mfccs_mean, mfccs = mfcc(data[0], sample[0])
print('MFCCs Mean:', mfccs_mean)
print('MFCCs shape:', mfccs.shape)
mfcc_v(mfccs,label[0])

In [ ]:
##Mel-spectrogram data[0]

mel_mean,mel=Mel(data[0],sample[0])
print('Mel Mean:',mel_mean)
print('Mel :',mel.shape)
mel_v(mel,label[0],sample[0])

In [ ]:
#zero_crossing data[0]

zcr=zero_crossing(data[0],sample[0])
print('Zcr:',zcr.shape)
zero_crossing_v(zcr,label[0],data[0],sample[0])

In [ ]:
code={}
x=0
for i in pd.unique(label):
    code[i]=x
    x+=1
pd.DataFrame(code.values(),columns=['Value'],index=code.keys())

In [ ]:
def get_Name(N):
    for x,y in code.items():
          if y==N:
                return x
for i in range(len(label)):
    label[i]=code[label[i]]
pd.DataFrame(label,columns=['Labels'])

### **Split Dataset**

In [ ]:
data=np.array(data).reshape(-1,16000,1)
label=np.array(label)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.1, random_state=44, shuffle =True)
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

### **Model**

In [ ]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics
from avalanche.models import SimpleMLP
from avalanche.training.supervised import Naive

# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model
model = SimpleMLP(num_classes=30)

# CL Benchmark Creation
perm_mnist = PermutedMNIST(n_experiences=3)
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, epoch_running=True,
                     experience=True, stream=True))

# Continual learning strategy
cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, evaluator=eval_plugin, device=device)

# train and test loop
results = []
for train_task in train_stream:
    cl_strategy.train(train_task, num_workers=4)
    results.append(cl_strategy.eval(test_stream))

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/evaluation.py:94: UserWarning: No loggers specified, metrics will not be logged
  warnings.warn("No loggers specified, metrics will not be logged")
/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in 

In [ ]:
print(results)

[{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.90625, 'Top1_RunningAcc_Epoch/train_phase/train_stream/Task000': 0.9246666666666666, 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9246666666666666, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9455, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.1345, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.1165, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.3988333333333333}, {'Top1_Acc_MB/train_phase/train_stream/Task000': 0.90625, 'Top1_RunningAcc_Epoch/train_phase/train_stream/Task000': 0.93405, 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.93405, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9249, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.9542, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.1376, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.6722333333333333}, {'Top1_Acc_MB/train_phase/train_stream/Task000': 0.8125, 'Top1_RunningAcc_Epoch/train_phase/t

In [ ]:
print(len(results))

3


In [ ]:
print(type(results))

<class 'list'>


In [ ]:
results[0]

{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.90625,
 'Top1_RunningAcc_Epoch/train_phase/train_stream/Task000': 0.9246666666666666,
 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9246666666666666,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9455,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.1345,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.1165,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.3988333333333333}

In [ ]:
results[1]

{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.90625,
 'Top1_RunningAcc_Epoch/train_phase/train_stream/Task000': 0.93405,
 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.93405,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9249,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.9542,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.1376,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.6722333333333333}

In [ ]:
results[2]

{'Top1_Acc_MB/train_phase/train_stream/Task000': 0.8125,
 'Top1_RunningAcc_Epoch/train_phase/train_stream/Task000': 0.9364666666666667,
 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9364666666666667,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9022,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.9439,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.956,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.9340333333333334}